Imports


In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

Load data

In [ ]:
data_path = "../dataset/states_socioeconomics.csv"
df = pd.read_csv(data_path)
state = df['State']

Predict clusters

In [ ]:
# Drop the state column
features = df.drop(['state'], axis=1)

start_year = 2012
end_year = 2022
df['cluster'] = pd.NA

num_clusters = 3
for year in range(start_year, end_year + 1):
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    year_features = features[features['year'] == year].drop(['year'], axis=1)
    cluster = kmeans.fit_predict(year_features)
    df.loc[df['year'] == year, 'cluster'] = cluster
    
df